# trajectory_compare_buoy

compare wave buoy path data with what parcels says

needs to run an entirely new parcels simulation such that the delta times match with the wave buoy's (or at least be close enough to it)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import utils
from parcels_utils import buoycsv_to_particleds
from plot_utils import plot_trajectories

In [ ]:
csv_path = utils.WAVEBUOY_DATA_DIR / "wavebuoy_5-01.csv"

In [ ]:
dataset = buoycsv_to_particleds(csv_path)
dataset

In [ ]:
filename = os.path.basename(csv_path)
nc_path = utils.WAVEBUOY_DATA_DIR / f"{os.path.splitext(filename)[0]}.nc"
dataset.to_netcdf(nc_path)

In [ ]:
plot_trajectories([nc_path])